In [1]:
import pandas as pd
import data_util
from tqdm.notebook import tqdm
#from tqdm import tqdm_notebook as tqdm
from data_generator import DataGenerator
from state_util import StateUtil
from tec_an import TecAn
import numpy as np
from data_util import *
from sklearn_model_hyper import *


import numpy as np
import math
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as scs
import scikitplot as skplt

from tensorflow.keras.layers import InputLayer, BatchNormalization, GlobalMaxPool1D, Bidirectional, Dense, Flatten, Conv2D, LeakyReLU, Dropout, LSTM, GRU, Input
from tensorflow.keras import Model, Sequential
from tensorflow.keras import datasets, layers, models
from tensorflow.keras import regularizers
from imblearn.over_sampling import RandomOverSampler
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
import tensorflow.keras as keras
import random
from catboost import CatBoost



from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, f1_score


In [2]:
import pandas as pd
import data_util
from tqdm import tqdm_notebook as tqdm
from data_generator import DataGenerator
from state_util import StateUtil
from tec_an import TecAn
import numpy as np



In [3]:
path = "./data/"
trainX_raw, trainY_raw = load_data("", "train", path)
valX_raw, valY_raw = load_data("", "Val", path)

trainX_balanced, trainY_balanced = get_balanced_set(trainX_raw, trainY_raw)

valX, valY = valX_raw, valY_raw

features = trainX_raw.shape[-1]


In [4]:
normalizer = tf.keras.layers.experimental.preprocessing.Normalization(input_shape=(features,))
normalizer.adapt(trainX_raw)

In [5]:
print(Counter(trainY_raw))


X_over, y_over = get_balanced_set(trainX_raw, trainY_raw)

print()
print("{} {}".format(Counter(y_over), Counter(trainY_raw)))

print("{} {} | {} {}".format(X_over.shape, y_over.shape, valX.shape, valY.shape))


Counter({0: 2165, 1: 1962})

Counter({0: 2165, 1: 2165}) Counter({0: 2165, 1: 1962})
(4330, 16) (4330,) | (459, 16) (459,)


In [18]:
%%time
from catboost import CatBoostClassifier
from sklearn.metrics import make_scorer, accuracy_score, f1_score


X_train, Y_train = trainX_raw, trainY_raw

x_norm = normalizer(trainX_raw).numpy()

clf = CatBoostClassifier()

params = {'iterations': [500],
          'depth': [
                    5,
                    6,
                    10
          ],
          #'loss_function': ['Logloss', 'CrossEntropy'],
          'l2_leaf_reg': np.logspace(-20, -19, 3),
          #'leaf_estimation_iterations': [10, 15, 20],
          'eval_metric': ['Precision'],
          #'use_best_model': ['True'],
          'logging_level':['Silent'],
          'random_seed': [42]
         }


CPU times: user 2.96 ms, sys: 2.52 ms, total: 5.48 ms
Wall time: 3.47 ms


In [19]:
%%time


clf_grid = gridSearch(X_train, Y_train, clf, params, make_scorer(precision_score))

#0.6743010531359825

{'depth': 10, 'eval_metric': 'Precision', 'iterations': 500, 'l2_leaf_reg': 1e-19, 'logging_level': 'Silent', 'random_seed': 42}
0.5170333974331347
CPU times: user 17min 34s, sys: 8min 6s, total: 25min 41s
Wall time: 6min 23s


In [20]:
#model = CatBoostClassifier(**clf_grid.best_params_)

#model.fit(X_train, Y_train)
#model.fit(x_norm, Y_train)

model = clf_grid.best_estimator_

eval_data(model, X_train, Y_train)

#5473554735547356

              precision    recall  f1-score   support

     class 0       1.00      1.00      1.00      2165
     class 1       1.00      1.00      1.00      1962

    accuracy                           1.00      4127
   macro avg       1.00      1.00      1.00      4127
weighted avg       1.00      1.00      1.00      4127



In [21]:
norm_val_x = normalizer(valX).numpy()

eval_data(model, valX, valY)


              precision    recall  f1-score   support

     class 0       0.51      0.60      0.55       219
     class 1       0.56      0.47      0.51       240

    accuracy                           0.53       459
   macro avg       0.54      0.53      0.53       459
weighted avg       0.54      0.53      0.53       459



In [17]:
from joblib import dump, load
dump(model, 'model/CatBoostClassifier') 

['model/CatBoostClassifier']

In [11]:
valX_raw, valY_raw = load_data("backtest", "train", path)

#eval_data(model, valX_raw, valY_raw)

for x in valX_raw:
    arrY = model.predict([x])
    if (arrY[0] == 1):
        print(np.array(x))

[ 3.71940412e+03  9.93627238e-01  9.34641748e+03 -4.74973300e+03
 -2.37489200e+03  9.86687888e-01  2.55461443e-01 -7.59324194e+00
 -5.80148247e+00  9.95256421e+01  8.75005484e+01  8.85088788e+01
  4.56945086e+01  8.85088788e+01  1.61035922e+04  1.62875529e+04]
[ 3.83323951e+03  9.89160712e-01  1.55569675e+04 -6.32187000e+03
 -3.16094400e+03  9.77327527e-01  3.39210513e-01 -7.87728089e+00
 -5.84039896e+00  9.90424802e+01  9.37502742e+01  8.19568565e+01
  5.29469502e+01  8.19568565e+01  1.60282088e+04  1.61558159e+04]
[ 3.67213176e+03  9.87582967e-01  2.33091461e+04 -7.89200750e+03
 -3.94592950e+03  9.74024969e-01  4.22435462e-01 -8.28154079e+00
 -5.88364552e+00  9.86794342e+01  9.68751371e+01  7.57149011e+01
  5.75004932e+01  7.57149011e+01  1.60022164e+04  1.61237333e+04]
[ 3.33981812e+03  9.86617977e-01  3.26024744e+04 -9.46130400e+03
 -4.73049050e+03  9.71999768e-01  5.05540838e-01 -8.97400108e+00
 -5.92947864e+00  9.83993214e+01  9.84375685e+01  6.98747515e+01
  5.99753677e+01  6.98

[-1.99121920e+01  5.24588846e-01  3.54556518e+00 -1.12074000e+02
 -4.09625000e+00  0.00000000e+00  1.00347638e+00 -7.03074253e+00
 -5.98092040e+00  2.12126562e+01  3.38809209e-19  3.04461953e-01
  3.68053695e-01  3.04461953e-01  3.46315078e-01 -1.93124612e-01]
[-2.64361273e+01  5.22471445e-01  4.38573206e+00  6.02820000e+01
 -3.79157500e+01  0.00000000e+00  1.00385451e+00 -7.18479246e+00
 -6.03211295e+00  1.86524465e+01  1.69404651e-19  2.62941657e-01
  3.47031435e-01  2.62941657e-01  4.13182121e-01 -5.18347513e-02]
[-3.43010676e+01  4.79828063e-01  5.08380583e+00  1.20115000e+02
 -6.01110000e+01  0.00000000e+00  1.00556047e+00 -7.36505195e+00
 -6.08411811e+00  1.51527290e+01  8.47031605e-20  2.11006511e-01
  3.19826377e-01  2.11006511e-01  4.56372024e-01 -2.51896246e-01]
[-4.57535753e+01  4.14980850e-01  5.58591844e+00  1.76861500e+02
 -1.35003750e+02  0.00000000e+00  1.00841238e+00 -7.58451898e+00
 -6.13760000e+00  9.91137426e+00  4.23516474e-20  1.38706097e-01
  2.83602559e-01  1.38

[-11.99072175   0.53838215   6.03771061  52.0445      75.8325
   0.19866461   1.00071243  -8.98849816  -6.28044796  10.8605009
   0.78123491   0.10482103   0.14236386   0.10482103   0.08074133
   0.1262137 ]
[ -4.96922806   0.55621688   4.46602194 -59.4155      86.523
   0.33636678   0.99950165  -6.50595657  -5.94634078  12.94679525
  78.16712356   0.11310563   0.12868424   0.11310563  -0.21170906
   1.02768629]
[-1.59106991e+01  4.58704276e-01  4.38810727e+00  1.76790000e+01
  5.70015000e+01  0.00000000e+00  1.00263826e+00 -6.79043401e+00
 -6.09728683e+00  6.12249245e+00  1.48172007e+01  5.09001054e-02
  1.00558505e-01  5.09001054e-02 -3.52495936e-01  8.86477275e-01]
[-2.09601539e+01  4.12771985e-01  4.24330909e+00  4.92005000e+01
  9.54705000e+01  0.00000000e+00  1.00402055e+00 -5.76707021e+00
 -6.15273269e+00  2.74630137e+00  7.31539451e+00  1.88779563e-02
  8.42790268e-02  1.88779563e-02 -2.39194947e-01  6.89451803e-01]
[-1.78575731e+01  4.96401116e-01  4.03811978e+00  6.26375000e+

[ 4.81929583e+00  5.57185142e-01 -2.62785598e-01  1.40775000e+01
 -1.00200000e+02  6.62926069e-01  9.98850822e-01 -6.80145296e+00
 -6.05423856e+00  6.25802341e+00  8.36092515e+01  6.20249907e-02
  4.71110295e-02  6.20249907e-02  5.72842179e-01 -2.70816516e-01]
[ 4.81981245e-01  4.90336540e-01  1.63976304e-02 -5.20765000e+01
 -1.27273250e+02  2.27309200e-01  1.00133000e+00 -6.91671905e+00
 -6.10578883e+00  4.97593579e+00  6.04549977e+01  4.89413475e-02
  4.74522416e-02  4.89413475e-02  2.67934917e-01 -9.56656442e-01]
[ 1.74619532e+00  5.37915790e-01  2.45662508e-01  2.37450000e+01
 -1.03922250e+02  4.84550454e-01  9.99610392e-01 -7.05225783e+00
 -6.16495464e+00  5.48963933e+00  3.07046996e+01  5.41032513e-02
  4.87048159e-02  5.41032513e-02  5.06274612e-01 -8.13164409e-01]
[-3.39464874e+00  4.72324690e-01  3.97519178e-01  7.39940000e+01
 -1.09233750e+02  8.52083765e-03  1.00222481e+00 -7.20363801e+00
 -6.22280876e+00  3.38661289e+00  1.65213065e+01  3.53537069e-02
  4.58644464e-02  3.53

[-1.05107876e+01  2.09938429e-01 -2.26576045e+01  5.65570000e+01
  9.07775000e+00  1.31470836e-01  1.01053561e+00 -5.73296677e+00
 -7.00593245e+00 -4.81889943e+01  4.17826753e-10 -6.77834629e-01
 -6.41991798e-01 -6.77834629e-01 -2.59064041e+00 -4.24650663e+00]
[-2.00569443e+01  2.00251617e-01 -2.28528865e+01 -8.91200000e+01
  3.94190000e+01  5.83662931e-02  1.01051193e+00 -5.73145309e+00
 -7.15856944e+00 -5.01907422e+01  1.72804546e-10 -7.25000715e-01
 -6.58461378e-01 -7.25000715e-01 -2.79514646e+00 -4.67113365e+00]
[-2.26183312e+01  2.06423907e-01 -2.30157871e+01 -6.85800000e+00
  9.60810000e+01  7.04416248e-02  1.00925143e+00 -5.77251207e+00
 -7.34112799e+00 -5.15271233e+01  8.85973222e-11 -7.52151688e-01
 -6.77212189e-01 -7.52151688e-01 -2.98702532e+00 -4.69726131e+00]
[-1.60393379e+01  2.71906727e-01 -2.31841134e+01 -2.82225000e+01
  2.00371750e+02  5.82448597e-01  1.00577183e+00 -5.81763186e+00
 -7.56683171e+00 -5.06115791e+01  5.04214033e-11 -7.44802065e-01
 -6.90810578e-01 -7.44

...

[ 6.59001084e+00  5.98826625e-01  1.60513686e+00 -2.61640000e+01
 -1.56881750e+02  7.42390702e-01  9.98949365e-01 -9.02187538e+00
 -5.84382155e+00  1.47509678e+01  9.99993275e+01  8.59109491e-02
  6.60040113e-02  8.59109491e-02  4.97890948e-01 -2.14575599e-01]
[ 5.25170818e+00  6.15758501e-01  1.95130940e+00 -2.31080000e+01
 -8.96167500e+01  9.10855648e-01  9.98742553e-01 -9.02249881e+00
 -5.80362298e+00  1.61342731e+01  9.99998379e+01  8.93307725e-02
  7.34814517e-02  8.93307725e-02  5.02850172e-01  6.46574754e-02]
[ 2.73236077e+00  5.75238916e-01  3.67555189e+00 -2.29925000e+01
 -2.84890000e+01  4.33225170e-02  1.00009664e+00 -9.02342478e+00
 -5.97890234e+00  2.73395075e+01  4.99999995e+01  1.30757363e-01
  1.22586401e-01  1.30757363e-01  3.41846120e-01  3.23582031e-01]
[ -1.52883753   0.54816293   3.87992246 -43.889      -13.36225
   0.           1.00060744  -8.3298613   -6.02727621  25.35951666
  24.99999978   0.11695595   0.12163366   0.11695595   0.28340329
   0.38417291]
[-3.735

[-7.69542438e+00  3.54552997e-01 -1.96461112e+00  4.28150000e+00
 -1.31556500e+02  0.00000000e+00  1.00245503e+00 -7.40901287e+00
 -6.45350143e+00 -1.51593850e+01  9.88006638e+00 -9.55157234e-02
 -7.22381786e-02 -9.55157234e-02  3.51166044e-02 -7.77260699e-01]
[-7.25485224e+00  3.92872827e-01 -1.93471978e+00  1.48025000e+01
 -1.30458500e+02  2.18630434e-01  1.00156843e+00 -7.63283587e+00
 -6.53422358e+00 -1.62491848e+01  5.09418533e+00 -9.99137883e-02
 -7.79604505e-02 -9.99137883e-02 -1.21589258e-01 -5.89511793e-01]
[-5.06211589e+00  3.57395315e-01 -2.10712743e+00  4.63035000e+01
  4.33585000e+01  2.24573877e-02  1.00217061e+00 -5.79885778e+00
 -6.93829206e+00 -1.79135794e+01  3.18633255e-01 -1.05613076e-01
 -9.02658384e-02 -1.05613076e-01 -4.61995005e-01 -8.90721072e-01]
[ -2.70200241   0.44020074  -2.25565772  46.424       37.327
   0.63822397   1.00034022  -5.84126979  -7.07341347 -17.03765681
   1.06091112  -0.1006197   -0.09240612  -0.1006197   -0.3041433
  -0.50445182]
[ -3.76127

[-1.77192078e+01  3.73034007e-01 -1.64419018e+00  1.67820000e+01
  1.24847000e+02  5.26496251e-01  1.00086952e+00 -5.86672206e+00
 -6.60432100e+00 -3.55042798e+01  2.44140623e-02 -2.17732009e-01
 -1.63228099e-01 -2.17732009e-01 -8.03242039e-01 -7.54258005e-02]
[  3.1231074    0.4390664   -3.70458416  -8.935      114.434
   0.91400757   0.99883115  -6.00786464  -6.92415537 -24.80512104
  58.26266634  -0.15759857  -0.16709327  -0.15759857  -0.65008286
  -0.27426078]
[ 1.13998665e+01  5.38753355e-01 -4.23676788e+00 -4.78240000e+01
  1.08978750e+02  1.00000000e+00  9.96349712e-01 -6.06177339e+00
 -7.06030222e+00 -1.93158838e+01  7.90951738e+01 -1.23504854e-01
 -1.58436233e-01 -1.23504854e-01 -4.04984405e-01  9.51714237e-02]
[ 1.64237415e+01  5.38250084e-01 -4.66092781e+00 -6.43825000e+01
  1.16858500e+02  9.98440141e-01  9.96605613e-01 -6.11582743e+00
 -7.21443972e+00 -1.49962053e+01  8.95588585e+01 -9.54241671e-02
 -1.45796041e-01 -9.54241671e-02 -4.39679931e-01 -3.56821184e-01]
[ 1.55808

[ 2.46802738e+00  4.78516780e-01 -1.95543694e+00 -8.67630000e+01
 -9.93250000e+00  4.29688741e-01  1.00121742e+00 -6.62916598e+00
 -6.03132898e+00  2.32653718e+00  9.13738834e+01  2.04242686e-02
  1.30026375e-02  2.04242686e-02 -6.97381249e-02 -3.28811010e-01]
[ 1.17541943e+00  5.01059304e-01 -1.58423436e+00 -5.65810000e+01
 -2.10530000e+01  3.69127314e-01  1.00082238e+00 -6.72515788e+00
 -6.08330400e+00  1.99522940e+00  8.05492408e+01  1.74657631e-02
  1.39261988e-02  1.74657631e-02 -9.96481139e-02 -2.69287355e-01]
[ 1.11277519e+00  5.14399773e-01 -1.22495874e+00 -6.81000000e-01
  1.48565000e+01  4.27789424e-01  1.00066974e+00 -6.83092013e+00
 -6.13777295e+00  2.02665323e+00  5.71702353e+01  1.81687901e-02
  1.48201246e-02  1.81687901e-02  5.46229471e-02 -3.82423576e-01]
[ 1.77655071e+00  4.89148841e-01 -3.68970491e-01  1.21952500e+02
 -1.36165500e+02  6.42608064e-02  1.00102195e+00 -7.23588748e+00
 -6.31959674e+00  3.58450846e+00  7.53479421e+00  2.78110569e-02
  2.24780003e-02  2.78

[   4.16087046    0.61583849    6.15349627  -46.2995     -132.3855
    0.38336767    0.99886867   -7.23321953   -5.98045656   23.16824096
   99.99999867    0.17870276    0.16626399    0.17870276    0.83077581
    0.56327509]
[-10.48777517   0.51192064   7.03531236  -5.748       -0.56725
   0.           1.00157004  -9.02267406  -6.25008534  17.51398422
  24.99999995   0.12180534   0.15353138   0.12180534   0.12408439
   0.56430773]
[-13.18241018   0.50656316   6.96696971   3.411       52.648
   0.           1.00160762  -9.02254858  -6.31449838  15.36510362
  12.49999998   0.10372983   0.14356781   0.10372983  -0.11202618
   0.53674809]
[-1.46075336e+01  5.06775498e-01  6.82874636e+00  7.93675000e+01
  1.21883500e+02  1.15819126e-03  1.00149701e+00 -9.02255311e+00
 -6.38349578e+00  1.37014643e+01  6.24999999e+00  8.84426706e-02
  1.32566122e-01  8.84426706e-02 -8.59850929e-02  5.91625486e-01]
[-1.38859399e+01  5.33205229e-01  6.61773308e+00  4.89655000e+01
  1.06620000e+02  1.54092504e-0

In [27]:
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectFromModel
from sklearn.svm import LinearSVC

#model = clf_grid.best_estimator_

clf = Pipeline([
  ('feature_selection', SelectFromModel(
      LinearSVC()
  )
  ),
  ('classification', CatBoostClassifier(logging_level = 'Silent'))
])

clf.fit(X_train, Y_train)



Pipeline(steps=[('classification',
                 <catboost.core.CatBoostClassifier object at 0x7f94d4e69400>)])

In [28]:
eval_data(clf, X_train, Y_train)

eval_data(clf, valX, valY)

              precision    recall  f1-score   support

     class 0       0.88      0.91      0.89      2165
     class 1       0.90      0.86      0.88      1962

    accuracy                           0.89      4127
   macro avg       0.89      0.89      0.89      4127
weighted avg       0.89      0.89      0.89      4127

              precision    recall  f1-score   support

     class 0       0.49      0.58      0.54       219
     class 1       0.55      0.45      0.50       240

    accuracy                           0.52       459
   macro avg       0.52      0.52      0.52       459
weighted avg       0.52      0.52      0.51       459

